In [ ]:
from resources.workspace import *
%matplotlib inline

# The (univariate) Kalman filter (KF)

The KF estimates the state variable (truth) $x$,  
which is assumed to "evolve" according to some linear "dynamics":
$$ x_{k} = F_{k-1} x_{k-1} + q_{k-1} \, , \qquad (\text{Dyn}) $$
where $q_{k-1} \sim \mathcal{N}(0, Q)$ is known dynamic process noise.  
Here, $k = 1,2,3,\ldots$ denotes the time indices.

The KF recursively (for increasing $k$) estimates $x_k$ .
It repeats a cycle consisting of two steps :
 
#### The forecast step
"propagates" the estimate (the pdf) of $x$.  
Note that (it may be shown that):  
if $x \sim \mathcal{N}(\mu, P)$ and $q \sim \mathcal{N}(0, Q)$  
then $F x + q \;\sim\; \mathcal{N}(F \mu, \; F^2P + Q)$  


Therefore, since $x_k$ obeys (Dyn),  
   if $\; p(x_{k-1}) = \mathcal{N}(x_{k-1} \mid \; \phantom{F_{k-1}}\hat{x}_{k-1}^a,\; \phantom{F_{k-1}^2} P_{k-1}^a) \, ,$
   for some $\hat{x}_{k-1}^a, \, P_{k-1}^a$,   
   then  $p(x_k)     = \mathcal{N}(x_k \; \; \, \mid \;  \underbrace{F_{k-1}
   \hat{x}_{k-1}^a}_{{\hat{x}_{k}}^f},\; \underbrace{F_{k-1}^2 P_{k-1}^a + Q}_{{P_k}^f}) \, .$  
   The KF forecast step amounts to the computation of the moments with superscript $f$. 

 
#### The analysis step
"updates" the prior/forecast into the posterior/"analysis", based on $p(y_k|x_k) = \mathcal{N}(y_k \mid x_k,R)$.  
 It was derived in [the previous tutorial](T2%20-%20Bayesian%20inference.ipynb#Gaussian-Gaussian-Bayes), with the symbolic translations in the below table. The KF analysis step amounts to the computation of the moments with superscript $a$. 
 
|         |                 |                 |         |         |
| ---     | ---             | ---             | ---     | ---     |
| Tut 2:  | $b$             | $B$             | $\mu$   | $P$     |
| Tut 3:  | $\hat{x}_{k}^f$ | $P_k^f$ | $\hat{x}_{k}^a$ | $P_k^a$ |

   

Note that this completes the cycle, which can then restart with the forecast from $k$ to $k+1$.


# A straight-line example

Many mathematical methods are tagged as "least squares". They typically have one thing in common: some sum of squared terms is being minimized. Both (least-squares) linear regression and Kalman filtering (KF) may be derived from "least squares".
Do they yield the same estimate (when applied to the linear regression problem)? That's what we'll investigate...

Consider the straight line
$$x_k = a k \, ,     \qquad \qquad (1) $$
and suppose we have observations ($y$) of the line, but corrupted by noise ($r$):
\begin{align*}
y_k &= x_k + r_k \, , \;\; \qquad (2)
\end{align*}
where $r_k \sim \mathcal{N}(0, R)$ for some $R>0$.

The code below sets up an experiment based on eqns. (1) and (2).  
Parameters:

In [ ]:
a = 0.4  # Slope (xx[k] = a*k) paramterer. Unknown to be estimated.
K = 10   # Length of experiment (final time index)
R = 1    # Observation noise strength

The following simulates a series of the truth ($x$) and observations ($y$).

In [ ]:
# Recall the naming convention: xx and yy hold time series of x and y.
xx = np.zeros(K+1) # truth states
yy = np.zeros(K+1) # obs

for k in 1+arange(K):
    xx[k] = a*k
    yy[k] = xx[k] + sqrt(R)*randn()

# The obs at k==0 should not be used (since we know xx[0]==0, it is worthless).
yy[0] = np.nan

### Esitmation by linear regression
(Least-squares) linear regression minimizes
$$J_K(\hat{a}) = \sum_{k=1}^K (y_k - \hat{a} k)^2 \, ,  \qquad (4)$$
yielding the estimator
$$\hat{a}_K = \frac{\sum_{k=1}^K {k} y_{k}}{\sum_{k=1}^K {k}^2} \, . \qquad \qquad (6)$$

**Exc 3.2:** Derive (6) from (4).

In [ ]:
#show_answer('LinReg deriv')

**Exc 3.4:** Program the linear regresson estimator (6)

In [ ]:
def lin_reg(k):
    "Liner regression estimator based on observations y_1, ..., y_k."
    ### INSERT ANSWER HERE ###
    return a

In [ ]:
#show_answer('LinReg_k')

Let's visualize the experiment:

In [ ]:
@interact(k=IntSlider(min=1, max=K))
def plot_experiment(k):
    plt.figure(figsize=(10,6))
    kk = arange(k+1)
    plt.plot(kk,xx[kk]        ,'k' ,label='true state ($x$)')
    plt.plot(kk,yy[kk]        ,'k*',label='noisy obs ($y$)')
    plt.plot(kk,kk*lin_reg(k) ,'r' ,label='Linear regress.')

    ### Uncomment this block AFTER doing the Exc 3.8 ###
    # pw_xxf, pw_xxa = weave_fa(xxf,xxa)
    # pw_kkf, pw_kka = weave_fa(arange(K+1))    
    # plt.plot(pw_kkf[:3*k],pw_xxf[:3*k] ,'c'  ,label='KF forecasts')
    # plt.plot(pw_kka[:3*k],pw_xxa[:3*k] ,'b'  ,label='KF analyses')
    # #plt.plot(kk,kk*xa[k]/k            ,'g--',label='KF extrapolated')

    plt.xlim([0,1.01*K])
    plt.ylim([-1,1.2*a*K])
    plt.xlabel('time index (k)')
    plt.ylabel('$x$, $y$, and $\hat{x}$')
    plt.legend(loc='upper left')
    plt.show()

---
### Estimation by the KF
In the following we tacke the same problem, but using the KF.

The observations equation (2)
yields the likelihood $p(y_k|x_k) = \mathcal{N}(y_k \mid x_k,R)$.  
Hopefully this is intuitive; otherwise, a more detailed derivation will be given in tutorial 4.

**Exc 3.6:** For the (univariate) KF,
we need to reformulate the problem of estimating the parameter $a$ 
as the problem of estimating $x_k$ (yielding $\hat{a}_k = \hat{x}_k / k$).
Derive and implement the "forecast model" $F_k$ (a function of $k>0$ only) such that
the recursion $x_{k+1} = F_k x_k$ is equivalent to eqn (1).

In [ ]:
def F(k):
    return ### INSERT ANSWER HERE ###

In [ ]:
#show_answer('Sequential 2 Recusive')

**Exc 3.7:** Explain why the KF is applicable to a larger class of problems than linear regression.

In [ ]:
#show_answer('LinReg ⊂ KF')

So although the KF (and its implementation below) may seem like "overkill" for this problem,
this "heavy machinery" can do a lot more, and will pay off later.

**Exc 3.8:** Implement the KF (described at the top of this notebook) in the below code block to estimate $x_k$ for $k=1,\ldots, K$.  

<div class="alert alert-warning" role="alert">
<b>NB:</b> for this example, do not use the "Kalman gain" form of the analysis update.
This problem involves the peculiar, unrealistic situation of infinities
(related to "improper priors") at `k==1`, yielding platform-dependent behaviour.
These peculariaties are of mainly of academic interest, and is not our focus here.
</div>

In [ ]:
Q = 0 # Dynamical model noise strength

# Allocation
xxa = np.zeros(K+1) # mean estimates -- analysis values (a)
xxf = np.zeros(K+1) # mean estimates -- forecast values (f)
PPa = np.zeros(K+1) # var  estimates -- analysis values (a)
PPf = np.zeros(K+1) # var  estimates -- forecast values (f)

def KF(k):
    "A cycle of the Kalman filter"
    # Forecast
    if k==1:
        PPf[k] = np.inf # The "initial" prior uncertainty is infinite...
        xxf[k] = 0      # ... thus the corresponding mean is inconsequential.
    else:
        PPf[k] = ### INSERT ANSWER HERE ###
        xxf[k] = ### INSERT ANSWER HERE ###
    # Analysis
    PPa[k] = ### INSERT ANSWER HERE ###
    xxa[k] = ### INSERT ANSWER HERE ###

# Run estimations/computations
for k in 1+arange(K):
    KF(k)

In [ ]:
#show_answer('KF_k')

**Exc 3.10:** Go back to the animation above and uncomment the block that plots the KF estimates.  
Visually: what is the relationship between the estimates provided by the KF and by linear regression?

In [ ]:
#show_answer('LinReg compare')

<em>Exercises marked with an asterisk (*) are optional.</em>

**Exc 3.12*:** This excercise proves (on paper) the result of the previous excercise.  
Note that the KF forecast step (here with $Q=0$) can be inserted in the analysis step, forming a single couple of recursions (and making the 'a' superscript unnecessary), which are recursive:

 * $\hat{x}_k = P_k \big(y_k/R \;+\; F_{k-1} \hat{x}_{k-1} / [F_{k-1}^2 P_{k-1}] \big) \qquad (11)$  
 * $P_k = 1/\big(1/R \;+\; 1/[F_{k-1}^2 P_{k-1}]\big) \qquad \qquad \quad (12)$

Now:

* (a). First show that $P_K = R\frac{K^2}{\sum_{k=1}^K k^2} \, . \;\;\; \qquad \quad (13)$
* (b). Then show that $\hat{x}_K = K\frac{\sum_{k=1}^K k y_k}{\sum_{k=1}^K k^2} = K \hat{a}_K$, where $\hat{a}_K$ is given by eqn (11).

In [ ]:
#show_answer('x_KF == x_LinReg')

#### Exc 3.14:
Let $x_{k+1} = F x_k$, *for a generic $F>1$ (note: $Q=0$)*. What does the sequence of $P_k$ converge to?  
You must start from eqn (12), because eqn (13) is for the straight-line example only.

In [ ]:
#show_answer('Asymptotic P when F>1')

#### Exc 3.15:
Redo Exc 3.14, but assuming  
 * (a) $F = 1$.
 * (b) $F < 1$.

In [ ]:
#show_answer('Asymptotic P when F=1')
#show_answer('Asymptotic P when F<1')

Thus, if $F>1$, the KF does not converge to 0 error. This is because, even though you keep gaining more information, this is balanced by the growth in uncertainty by the forecast. On the other hand, if $F \leq 1$ then the error converges to zero.

In general, however, $F$ depends on $k$, as will the other system matrices ($Q, R$).
Then, there is no limit value that the state distribution (and its parameters) converges to.

**Exc 3.18*:** Set $Q$ to 1 or more. Re-compute the KF estimates. By interpreting the meaning of $Q$, explain why the KF estimate is now closer to the obs (always at the latest time instance) than the linear regression estimate.

**Exc 3.20*:** Now change $R$. The KF estimates should not change (in this particular example). Why?

---

# A higher-order example
Above we saw that the KF produces reasonable results for straight lines (in so far as linear regression does!).
What about more intricate time series?

### The model
Note that the straight line (eqn 1 at the top) could result from discretizing the model
\begin{align*}
\frac{dx}{dt} &= a \, , \\
x(0) &= 0 \, ,
\end{align*}
using `dt = 1`.
Now, instead, we're going to consider the model
$$ \frac{d^M x}{dt^M} = 0 \, .$$

This can be written as 1-st order vector (i.e. coupled system of) ODE:
$$ \frac{d x^m}{dt} = x^{m+1} \, , \quad \frac{d x^M}{dt} = 0 \, ,$$
where the superscript $m = 1,\ldots,M$ is the index of the state vector element.

To make it more interesting, we'll add two terms to this evolution model:  
 - damping: $\beta x^m$, with $\beta < 0$;
 - noise: $\frac{d q^m}{dt}$.  

Thus,
$$ \frac{d x^m}{dt} = \beta x^m + x^{m+1} + \frac{d q^m}{dt} \, ,$$
where $q^m$ is the noise process, and $\beta = \log(0.9)$.

Discretized, with a time step `dt=1`, this yields
$$ x^m_{k+1} = 0.9 x^m_k + x^{m+1}_k + q^m_k\, ,$$

In summary, $\mathbf{x}_{k+1} = \mathbf{F} \mathbf{x}_k + \mathbf{q}_k$, with $\mathbf{F}$ as below.

In [ ]:
M = 4 # model order (and also ndim)
F_matrix = 0.9*eye(M) + diag(ones(M-1),1)
print(F_matrix)

### Estimation by the Kalman filter (and smoother) with DAPPER

Note that this is an $M$-dimensional time series. 
However, we'll only observe the first (0th) component.

We shall not write the code for the multivariate Kalman filter,
because it already exists in DAPPER in `da_methods.py` and is called `ExtKF()`.

The following code configures an experiment based on the above model. Don't worry about the specifics. We'll get back to how to use DAPPER later.


In [ ]:
# Forecast dynamics
Dyn = linear_model_setup(F_matrix)
Dyn['noise'] = 0.0001*(1+arange(M))

# Initial conditions
X0 = GaussRV(m=M,C=0.02*arange(M))

# Observe 0th component only
Obs = partial_direct_obs_setup(M,[0])
Obs['noise'] = 1000

# Time settings
t = Chronology(dt=1,dtObs=5,K=250)

# Wrap-up
HMM = HiddenMarkovModel(Dyn,Obs,t,X0)

This generates (simulates) a synthetic truth (xx) and observations (yy)

In [ ]:
xx,yy = simulate(HMM)

In [ ]:
plt.figure(figsize=(6,4))
for m,x in enumerate(xx.T):
    plt.plot(x,label="x^%d"%m)
plt.legend();

Now we'll run assimilation methods on the data. Firstly, the KF, available as `ExtKF` in DAPPER:

In [ ]:
stats_KF = ExtKF (store_u=1).assimilate(HMM,xx,yy)

We'll also run the "Kalman smoother" available as `ExtRTS`.
Without going into details, this method is based on the Kalman *filter* but,
being a *smoother*,
it also goes backwards and updates previous estimates with future (relatively speaking) observations.

In [ ]:
stats_KS = ExtRTS(store_u=1).assimilate(HMM,xx,yy)

### Estimation by "time series analysis"
The following methods perform time series analysis of the observations, and are mainly derived from signal processing theory.
Considering that derivatives can be approximated by differentials, it is plausible that the above model could also be written as an AR(M) process. Thus these methods should perform quite well.

In [ ]:
# Tools
import scipy.signal as sp_sp
normalize = lambda x: x / x.sum()
truncate  = lambda x,n: np.hstack([x[:n],zeros(len(x)-n)])

# We only estimate the 0-th component.
signal = yy[:,0]

# Estimated signals
ESig = {} 
ESig['Gaussian']       = sp_sp.convolve(signal, normalize(sp.signal.gaussian(30,3)),'same')
ESig['Wiener']         = sp_sp.wiener(signal)
ESig['Butter']         = sp_sp.filtfilt(*sp_sp.butter(10, 0.12), signal, padlen=len(signal)//10)
ESig['Spline']         = sp.interpolate.UnivariateSpline(t.kkObs,signal,s=1e4)(t.kkObs)
ESig['Trunc. Fourier'] = np.fft.irfft(truncate(np.fft.rfft(signal),len(signal)//14))

### Comparison
The following code plots the results.

<div class="alert alert-warning" role="alert">
<b>NB:</b> The GUI is buggy; it might be necessary to execute the cell multiple times. 
</div>

In [ ]:
%matplotlib notebook

@interact(Visible=SelectMultiple(options=['Truth','Kalman smoother','Kalman filter','My Method']+list(ESig)))
def plot_results(Visible):
    plt.figure(figsize=(9,5))
    plt.plot(t.kkObs,yy,'k.',alpha=0.4,label="Obs")
    if 'Truth'           in Visible: plt.plot(t.kk   ,xx[:,0]           ,'k',label="Truth")
    if 'Kalman smoother' in Visible: plt.plot(t.kk   ,stats_KS.mu.u[:,0],'m',label="K. smoother")
    #if'Kalman filter u' in Visible: plt.plot(t.kk   ,stats_KF.mu.u[:,0],'b',label="K. filter (u)")
    #if'Kalman filter f' in Visible: plt.plot(t.kkObs,stats_KF.mu.f[:,0],'b',label="K. filter (f)")
    #if'Kalman filter a' in Visible: plt.plot(t.kkObs,stats_KF.mu.a[:,0],'b',label="K. filter (a)")
    if 'Kalman filter'   in Visible:
        pw_xxf, pw_xxa = weave_fa(stats_KF.mu.f[:,0],stats_KF.mu.a[:,0])
        pw_kkf, pw_kka = weave_fa(t.kkObs)
        plt.plot(pw_kkf,pw_xxf,'b',label="KF. forecast")
        plt.plot(pw_kka,pw_xxa,'c',label="KF. analysis")
    
    if 'My Method' in Visible and 'stats_MM' in locals():
        pw_xxf, pw_xxa = weave_fa(stats_MM.mu.f[:,0],stats_MM.mu.a[:,0])
        pw_kkf, pw_kka = weave_fa(t.kkObs)
        plt.plot(pw_kkf,pw_xxf,'y',label=stats_MM.config.da_method.__name__+" forecast")
        plt.plot(pw_kka,pw_xxa,'g',label=stats_MM.config.da_method.__name__+" analysis")
    
    for method, estimate in ESig.items():
        if method in Visible: plt.plot(t.kkObs, estimate,label=method)
    
    plt.ylabel('$x^0$, $y$, and $\hat{x}^0$')
    plt.xlabel('Time index ($k$)')
    plt.legend()
    plt.show()

Visually, it's hard to imagine better performance than from the Kalman smoother.
However, recall the advantage of the Kalman filter (and smoother): *they know the forecast model that generated the truth*.

Since the noise levels Q and R are given to the DA methods (but they don't know the actual outcomes/realizations of the random noises), they also do not need any *tuning*, compared to signal processing filters, or choosing between the myriad of signal processing filters [out there](https://docs.scipy.org/doc/scipy/reference/signal.html#module-scipy.signal).

In [ ]:
def average_error(estimate_at_obs_times):
    return np.mean(np.abs(xx[t.kkObs,0] - estimate_at_obs_times))

for method, estimate in ESig.items():
    print(method   , average_error(estimate))
print('K. smoother', average_error(stats_KS.mu.u[t.kkObs,0]))
print('K. filter'  , average_error(stats_KF.mu.a[:,0]))
# print('My Method', average_error(stats_MM.mu.a[:,0])) # uncomment after Exc 3.28

**Exc 3.22:** Theoretically, in the long run, the Kalman smoother should yield the optimal result. Verify this by increasing the experiment length to K=10**4.

**Exc 3.24:** Re-run the experiment with different paramters, for example the observation noise strength or `dkObs`.  
[Results will differ even if you changed nothing because the truth noises (and obs) are stochastic.]

**Exc 3.26:** Right before executing the assimilations (but after simulating the truth and obs), change $R$ by inserting:

    HMM.h.noise = GaussRV(C=0.01*eye(1))
    
What happens to the estimates of the Kalman filter and smoother?

**Exc 3.28*:** Try out different methods from DAPPER by replacing `MyMethod` below with one of the following:
 - Climatology
 - Var3D
 - OptInterp
 - EnKF
 - EnKS
 - PartFilt

You typically also need to set (and possibly tune) some method parameters. Otherwise you will get an error (or possibly the method will perform very badly). You may find (some) documentation for each method in its source code...

In [ ]:
stats_MM = MyMethod(param1=val1,...).assimilate(HMM,xx,yy)

### Summary
Like linear regression, time series analysis is also a subset of  data assimilation [(much of time series analysis can be formulated as state estimation)](https://www.google.com/search?q="We+now+demonstrate+how+to+put+these+models+into+state+space+form"). Moreover, DA methods produce uncertainty quantification, something which is usually more obscure with time series analysis methods.

Still, the best is yet to come: DA methods should have the capacity to handle inhomogeneous, multivariate, sparsely observed, chaotic systems (which is more fun than stochastically-driven signals such as the above example). This is what we'll get to next.

### Next: [Dynamical systems, chaos, Lorenz](T4 - Dynamical systems, chaos, Lorenz.ipynb)